<a href="https://colab.research.google.com/github/The-Gupta/TED-Scraper/blob/master/Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 22-JUN-20  2250.

In [1]:
import requests, bs4, sys, os, time, json, re, random, ast
from multiprocessing import Process, Manager
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from datetime import datetime


'''  IGNORING WARNINGS  '''
import warnings
warnings.filterwarnings('ignore')


'''  NOTEBOOK DISPLAY SETTINGS  '''
pd.options.display.max_rows = 10000    # None
pd.options.display.max_columns = None

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.expand_frame_repr', False)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
# Using Google Drive for storage.
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
cd "/content/gdrive/My Drive/Vishal/PersonalProject/TED/6/"

/content/gdrive/My Drive/Vishal/PersonalProject/TED/6


# Download Meta and Transcript

In [5]:
time__  =  time.time()





urls = []
page_number=0

while 1:
    page_number += 1

    res  =  requests.get("https://www.ted.com/talks?sort=popular&page=" + str(page_number), 
                         headers = {'User-agent': 'your bot 0.1'})

    soup = bs4.BeautifulSoup(res.text)
    e = soup.select("div.container.results div.col")
    
    if len(e) == 0:    break    # No more videos.
    
    for  u  in e:
        urls.append(  "https://www.ted.com" + u.select("div.media__image a.ga-link")[0].get("href")  )


# Saving.
f = open('TED_Talk_URLs.txt', 'w')
f.write('\n'.join(urls))
f.close()



def download(urls, id_, csv_list):
                for count, url in enumerate(urls):

                        def get_transcript(url):
                                transcript = ""
                                transcript_res = requests.get(url, headers = {'User-agent': 'your bot 0.1'})
                                
                                soup = BeautifulSoup(transcript_res.text)
                                e = soup.select('div.Grid.Grid--with-gutter.p-b:4')

                                for  e_  in e:
                                    classes = e_.get('class')
                                    text = e_.select('p')[0].text
                                    transcript += text.strip().replace('\t', '').replace('\n', ' ')
                                
                                if (transcript_res.status_code!=200) or (transcript_res.text=='') or (transcript==''):
                                    count_=0
                                    while  count_ < 3:    # Check 3 more times
                                        time.sleep(random.randint(0,900)/1000)     # Randomly wait for 0-0.9 seconds.
                                        transcript_res = requests.get(url, headers = {'User-agent': 'your bot 0.1'})

                                        soup = BeautifulSoup(transcript_res.text)
                                        e = soup.select('div.Grid.Grid--with-gutter.p-b:4')

                                        for  e_  in e:
                                            classes = e_.get('class')
                                            text = e_.select('p')[0].text
                                            transcript += text.strip().replace('\t', '').replace('\n', ' ')

                                        count_ += 1
                                        if (transcript_res.status_code==200) and (transcript_res.text!='') and (transcript!=''):    break

                                return transcript



                        def get__json_obj(url):
                            res = requests.get(url.strip(), headers = {'User-agent': 'your bot 0.1'})
                            html = res.text

                            start_index  =  html.find('<script data-spec="q">q("talkPage.init",')
                            end_index    =  html[start_index:].find(')</script>')
                            script_tag   =  html[start_index: start_index + end_index]
                            json_obj  =  script_tag[len('<script data-spec="q">q("talkPage.init",'):]
                            return json_obj

                        json_obj  =  get__json_obj(url)

                        if not json_obj:
                            count=0
                            while  count < 3:    # Check 3 more times
                                json_obj  =  get__json_obj(url)
                                count += 1
                                if json_obj:    break

                        if not json_obj:    print(url);continue;
                        else:               metadata = json.loads(json_obj)["__INITIAL_DATA__"]



                        def get_value(l, m=metadata):
                            for i in l:
                                try:    m = m[i]
                                except: return ''
                            return m





                        def html_to_text(html):
                            if str(html)!='nan':
                                soup = BeautifulSoup(html)
                                return soup.get_text()
                            else: return html


                        d = dict()

                        # ALL THESE SPEAKER ATTRIBUTES CORRESPOND TO THE 1ST SPEAKER.
                        d["talk__id"]  =  get_value(["current_talk"], metadata)
                        d["talk__name"]  =  get_value(["talks", 0, "title"], metadata)
                        d["talk__description"]  =  get_value(["description"], metadata)


                        d["view_count"]  =  get_value(["viewed_count"], metadata)
                        d["comment_count"]  =  get_value(["comments", "count"], metadata)


                        d["duration"]  =  get_value(["talks", 0, "duration"], metadata)    # In seconds.


                        language  =  get_value(["language"], metadata)
                        url__transcript  =  url + "/transcript?language=" + language
                        d["transcript"]  =  get_transcript(url__transcript)


                        d["video_type_name"]  =  get_value(["talks", 0, "video_type", "name"], metadata)    # One of:  TED Stage Talk, TEDx Talk, TED-Ed Original, TED Institute Talk, Best of Web, Original Content, TED Salon Talk (partner), Custom sponsored content
                        d["event"]  =  get_value(["event"], metadata)
                        

                        d["number_of__speakers"]  =  len(get_value(["speakers"], metadata) or "")

                        # First speaker details.
                        d["speaker__id"]  =  get_value(["speakers", 0, "id"], metadata)                        
                        d["speaker__name"]  =  get_value(["talks", 0, "speaker_name"], metadata)
                        d["speaker__description"]  =  get_value(["speakers", 0, "description"], metadata)
                        d["speaker__who_he_is"]  =  get_value(["speakers", 0, "whotheyare"], metadata)

                        d["speaker__why_listen"]  =  html_to_text(get_value(["speakers", 0, "whylisten"], metadata))

                        d["speaker__what_others_say"]  =  get_value(["speakers", 0, "whatotherssay"], metadata)
                        d["speaker__is_published"]  =  get_value(["speakers", 0, "is_published"], metadata)
                        
                        d["all_speakers_details"]  =  get_value(["speakers"], metadata)
                        

                        d["is_talk_featured"]  =  get_value(["talks", 0, "is_featured"], metadata)
                        d["has_talk_citation"]  =  get_value(["talks", 0, "has_citations"], metadata)
                        

                        # Recorded and Published time.
                        temp  =  get_value(["talks", 0, "recorded_at"], metadata)
                        d["recording_date"]  =  temp  if temp==None  else temp[:10]
                        
                        t  =  get_value(["talks", 0, "player_talks", 0, "published"], metadata)
                        d["published_timestamp"]  =  datetime.utcfromtimestamp(int(t)).strftime('%Y-%m-%d %H:%M:%S')
                        

                        # Tags
                        d["talks__tags"]  =  get_value(["talks", 0, "tags"], metadata)
                        d["number_of__tags"]  =  len(get_value(["talks", 0, "tags"], metadata) or "")
                        

                        d["language"]  =  language
                        d["native_language"]  =  get_value(["talks", 0, "player_talks", 0, "nativeLanguage"], metadata)
                        d["language_swap"]  =  get_value(["language_swap"], metadata)
                                                
                        d["is_subtitle_required"]  =  get_value(["talks", 0, "player_talks", 0, "isSubtitleRequired"], metadata)
                        

                        # URLs.
                        d["url__webpage"]  =  get_value(["url"], metadata)
                        d["url__audio"]  =  get_value(["talks", 0, "downloads", "audioDownload"], metadata)
                        d["url__video"]  =  get_value(["talks", 0, "downloads", "nativeDownloads"], metadata)
                        d["url__photo__talk"]  =  get_value(["talks", 0, "hero"], metadata)
                        d["url__photo__speaker"]  =  get_value(["speakers", 0, "photo_url"], metadata)
                        
                        d["url__subtitled_videos"]  =  get_value(["talks", 0, "downloads", "subtitledDownloads"], metadata)
                        d["number_of__subtitled_videos"]  =  len(get_value(["talks", 0, "downloads", "subtitledDownloads"], metadata) or "")
                        
                        d["talk__download_languages"]  =  get_value(["talks", 0, "downloads", "languages"], metadata)    # [?]
                        d["number_of__talk__download_languages"]  =  len(get_value(["talks", 0, "downloads", "languages"], metadata) or "")    # [?]
                        

                        # More resources.
                        d["talk__more_resources"]  =  get_value(["talks", 0, "more_resources"], metadata)
                        d["number_of__talk__more_resources"]  =  len(get_value(["talks", 0, "more_resources"], metadata) or "")


                        # Recommendations.
                        d["talk__recommendations__blurb"]  =  get_value(["talks", 0, "recommendations", "blurb"], metadata)
                        
                        d["talk__recommendations"]  =  get_value(["talks", 0, "recommendations", "rec_lists"], metadata)
                        d["number_of__talk__recommendations"]  =  len(get_value(["talks", 0, "recommendations", "rec_lists"], metadata) or "")


                        # Related Talks.
                        d["related_talks"]  =  get_value(["talks", 0, "related_talks"], metadata)
                        d["number_of__related_talks"]  =  len(get_value(["talks", 0, "related_talks"], metadata) or "")


                        # Durations.
                        d["intro_duration"]  =  get_value(["talks", 0, "player_talks", 0, "introDuration"], metadata)    # [?]
                        d["ad_duration"]  =  get_value(["talks", 0, "player_talks", 0, "adDuration"], metadata)    # [?]
                        d["post_ad_duration"]  =  get_value(["talks", 0, "player_talks", 0, "postAdDuration"], metadata)    # [?]
                        d["external__duration"]  =  get_value(["talks", 0, "player_talks", 0, "external", "duration"], metadata)    # [?]
                        d["external__start_time"]  =  get_value(["talks", 0, "player_talks", 0, "external", "start_time"], metadata)    # [?]
                        

                        d["talks__player_talks__resources__h264__00__bitrate"]  =  get_value(["talks", 0, "player_talks", 0, "resources", "h264", 0, "bitrate"], metadata)    # [?]


                        # [?] Take Action.
                        d["talks__take_action"]  =  get_value(["talks", 0, "take_action"], metadata)
                        d["number_of__talks__take_actions"]  =  len(get_value(["talks", 0, "take_action"], metadata) or "")


                        csv_list.append(d)



csv_list_ = []
with  Manager()  as manager:
        csv_list = manager.list()    # SPECIAL variable - can be used only locally.
        Processess = []


        concurreny_count  =  100
        urls_  =  [urls[(i* (len(urls)//concurreny_count)):((i+1)* (len(urls)//concurreny_count))]    for i in range(concurreny_count)]

        leftovers  =  urls[(concurreny_count * (len(urls)//concurreny_count))  :  len(urls)]
        for i in range(len(leftovers)):    urls_[i] += [leftovers[i]]


        for  (id_,urls__)  in enumerate(urls_):
            p = Process(target=download, args=(urls__,id_,csv_list))
            Processess.append(p)
            p.start()


        # block until all the threads finish (i.e. block until all **download** function calls finish)
        for t in Processess:    t.join()
        
        csv_list_ = list(csv_list)



# Creating DataFrame.
df  =  pd.DataFrame(csv_list_)

# Sort - most popular first.
df = df.sort_values("view_count", ascending=False)

# Saving.
df.to_excel('TED_Talk.xlsx', encoding='utf-8', index=False)
df.to_csv('TED_Talk.csv', index=False, encoding='utf-8')




print(round(time.time()  -  time__))

304


### Quick look at the data.

In [6]:
df  =  pd.read_excel('TED_Talk.xlsx')

In [7]:
df.shape[0], df[~df['transcript'].isna()].shape[0]

(4609, 4091)

In [8]:
# Look at the first row.
df.iloc[0]

talk__id                                             66                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

### 'Column Name', 'Unique Count', 'Missing Count'

In [9]:
pd.DataFrame([[col, df[col].nunique(), df[col].isna().sum()]  for  col  in df], columns = ['Column Name', 'Unique Count', 'Missing Count'])

,Column Name,Unique Count,Missing Count
0,talk__id,4609,0
1,talk__name,4609,0
2,talk__description,4609,0
3,view_count,4409,0
4,comment_count,601,1144
5,duration,1238,0
6,transcript,4091,518
7,video_type_name,8,0
8,event,550,0
9,number_of__speakers,7,0


### How to access a NESTED column value.

In [10]:
# Accessing a list.
x = ast.literal_eval(df['talks__tags'].iloc[0])
x, x[-2]

(['children',
  'creativity',
  'culture',
  'dance',
  'education',
  'parenting',
  'teaching'],
 'parenting')

In [11]:
# Accessing a list of dictionaries.
x = ast.literal_eval(df['related_talks'].iloc[0])
x, x[0], x[1]['slug']

([{'duration': 1008,
   'hero': 'https://pe.tedcdn.com/images/ted/172559_800x600.jpg',
   'id': '865',
   'slug': 'sir_ken_robinson_bring_on_the_learning_revolution',
   'speaker': 'Sir Ken Robinson',
   'title': 'Bring on the learning revolution!',
   'viewed_count': 9404739},
  {'duration': 1151,
   'hero': 'https://pe.tedcdn.com/images/ted/de98b161ad1434910ff4b56c89de71af04b8b873_1600x1200.jpg',
   'id': '1738',
   'slug': 'sir_ken_robinson_how_to_escape_education_s_death_valley',
   'speaker': 'Sir Ken Robinson',
   'title': "How to escape education's death valley",
   'viewed_count': 10179741},
  {'duration': 1027,
   'hero': 'https://pe.tedcdn.com/images/ted/3821f3728e0b755c7b9aea2e69cc093eca41abe1_2880x1620.jpg',
   'id': '2276',
   'slug': 'linda_cliatt_wayman_how_to_fix_a_broken_school_lead_fearlessly_love_hard',
   'speaker': 'Linda Cliatt-Wayman',
   'title': 'How to fix a broken school? Lead fearlessly, love hard',
   'viewed_count': 2176523},
  {'duration': 1302,
   'hero'

In [12]:
# Accessing a dictionary of dictionary.
x = ast.literal_eval(df['url__subtitled_videos'].iloc[0])
x, x['az'], x['az']['name']

({'af': {'high': 'https://download.ted.com/talks/SirKenRobinson_2006-480p-af.mp4?apikey=acme-roadrunner',
   'low': 'https://download.ted.com/talks/SirKenRobinson_2006-low-af.mp4?apikey=acme-roadrunner',
   'name': 'Afrikaans'},
  'ar': {'high': 'https://download.ted.com/talks/SirKenRobinson_2006-480p-ar.mp4?apikey=acme-roadrunner',
   'low': 'https://download.ted.com/talks/SirKenRobinson_2006-low-ar.mp4?apikey=acme-roadrunner',
   'name': 'Arabic'},
  'az': {'high': 'https://download.ted.com/talks/SirKenRobinson_2006-480p-az.mp4?apikey=acme-roadrunner',
   'low': 'https://download.ted.com/talks/SirKenRobinson_2006-low-az.mp4?apikey=acme-roadrunner',
   'name': 'Azerbaijani'},
  'be': {'high': 'https://download.ted.com/talks/SirKenRobinson_2006-480p-be.mp4?apikey=acme-roadrunner',
   'low': 'https://download.ted.com/talks/SirKenRobinson_2006-low-be.mp4?apikey=acme-roadrunner',
   'name': 'Belarusian'},
  'bg': {'high': 'https://download.ted.com/talks/SirKenRobinson_2006-480p-bg.mp4?api

# Download Media Files.

In [4]:
def download_in_parallel(concurreny_count, urls, fn, folder, extension):
        Processes = []

        urls_  =  [urls[(i* (len(urls)//concurreny_count)):((i+1)* (len(urls)//concurreny_count))]    for i in range(concurreny_count)]

        leftovers  =  urls[(concurreny_count * (len(urls)//concurreny_count))  :  len(urls)]
        for i in range(len(leftovers)):    urls_[i] += [leftovers[i]]


        for  (id_,urls__)  in enumerate(urls_):
            p = Process(target=fn, args=(urls__,id_, folder, extension))
            Processes.append(p)
            p.start()


        # block until all the threads finish (i.e. block until all function_x calls finish)
        for t in Processes:    t.join()



def download_any(urls, id_, folder, extension):    # To download any file.
    for  (count, url) in enumerate(urls):
            url,filename = url
            try:                      urllib.request.urlretrieve(url, folder+'/'+str(filename)+extension)
            except Exception as e:    print('\n\n\n', url, '\n', e)

In [5]:
df  =  pd.read_excel('TED_Talk.xlsx')

In [6]:
# DOWNLOADING 'url__photo__talk'. Saving by 'talk__id'.

urls  =  df[['url__photo__talk', 'talk__id']].dropna().values.tolist()

folder, extension  =  'PHOTO__TALK', '.jpg'
if not os.path.exists(folder):    os.makedirs(folder)


time__  =  time.time()

concurreny_count  =  100
download_in_parallel(concurreny_count, urls, download_any, folder, extension)

print(round(time.time()  -  time__))

294


In [16]:
# DOWNLOADING 'url__photo__speaker'. Saving by 'speaker__id'.

urls  =  [[i[0], int(i[1])]  for i in df[['url__photo__speaker', 'speaker__id']].dropna().values.tolist()]

folder, extension  =  'PHOTO__SPEAKER', '.jpg'
if not os.path.exists(folder):    os.makedirs(folder)


time__  =  time.time()

concurreny_count  =  100
download_in_parallel(concurreny_count, urls, download_any, folder, extension)

print(round(time.time()  -  time__))

36


In [17]:
# DOWNLOADING 'url_audio'. Saving by 'talk__id'.

urls  =  df[['url__audio', 'talk__id']].dropna().values.tolist()

folder, extension  =  'AUDIO', '.mp3'
if not os.path.exists(folder):    os.makedirs(folder)


time__  =  time.time()

concurreny_count  =  100
download_in_parallel(concurreny_count, urls, download_any, folder, extension)

print(round(time.time()  -  time__))

399


In [16]:
# DOWNLOADING 'url_video'. Saving by 'talk__id'.

s = df[['url__video', 'talk__id']].dropna().values.tolist()

def get_url(i):    # Because sometime 'high' is None.
    if ast.literal_eval(i)['high']:        return ast.literal_eval(i)['high']
    elif ast.literal_eval(i)['medium']:    return ast.literal_eval(i)['medium']
    else:                                  return ast.literal_eval(i)['low']

urls = [[get_url(i[0]), i[1]]  for i in s]

folder, extension  =  'VIDEO', '.mp4'
if not os.path.exists(folder):    os.makedirs(folder)



time__  =  time.time()

concurreny_count  =  8
download_in_parallel(concurreny_count, urls, download_any, folder, extension)

print(round(time.time()  -  time__))

5562
